In [22]:
import boto3

REGION = boto3.Session().region_name

# Replace with your S3 bucket where you prepared the IMDB data for Neptune use
SOURCE_S3_PATH_NOSLASH=f"s3://ekg-da-poc"

# Replace with your Amazon OpenSearch Service domain endpoint
FTS_AOS_ENDPOINT="https://z9urozgne3v5p4ywe2fj.us-east-1.aoss.amazonaws.com"

VS_AOS_ENDPOINT="https://8kd01utb7q4h0s0lec3g.us-east-1.aoss.amazonaws.com"

# Replace with your IAM role for the GNN embedding pipeline.
SAGEMAKER_ROLE="arn:aws:iam::196856463470:role/SandboxServiceRole"


In [23]:
!pip install -q opensearch-py

In [24]:
import boto3
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

service = 'aoss'
region2 = 'us-east-1'

credentials = boto3.Session().get_credentials()
auth2 = AWSV4SignerAuth(credentials, region2, service)

vs_aos_client = OpenSearch(
    hosts = [{'host': VS_AOS_ENDPOINT.split("//")[1], 'port': 443}],
    http_auth=auth2,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize=20,
)

In [25]:
vs_aos_client.indices.get_alias(index="*")

{'movie_sentence': {'aliases': {}},
 'movie_2': {'aliases': {}},
 'movie_gnn_2': {'aliases': {}},
 'movie': {'aliases': {}},
 '.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}},
 'movie_gnn': {'aliases': {}},
 'movie_sentence_2': {'aliases': {}},
 'python-test-index': {'aliases': {}},
 'books-index': {'aliases': {}}}

In [26]:
!pip install sentence-transformers

In [27]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(sentences):
    embeddings = model.encode(sentences)
    return embeddings

def get_str_embedding(embedding):
    return ';'.join([str(x) for x in embedding])

def parse_str_embedding(s):
    toks= s.split(";")
    return list(map(lambda x: float(x), toks))

In [28]:
import csv

def csv_open(fname, headers):
    ff = []
    ff.append(open(fname, "w"))
    ff.append(csv.writer(ff[0]))
    csv_write(ff, headers)
    return ff

def csv_close(ff):
    ff[0].close()

def csv_write(ff, arr):
    ff[1].writerow("" if pd.isna(a) or a=='-' else a for a in arr)


In [29]:
!pip install graph-notebook --upgrade

In [ ]:
%load_ext graph_notebook.magics

In [14]:
%sparql

UsageError: Line magic function `%sparql` not found.


In [15]:
%%bash -s "$SOURCE_S3_PATH_NOSLASH" 

echo Download GNN
aws s3 cp $1/embeddings/movie.csv gnn_embeddings.csv


Download GNN
download: s3://ekg-da-poc/embeddings/movie.csv to ./gnn_embeddings.csv


In [17]:
!ls -altr ./*

-rw-rw-rw- 1 root root       5989 Jun 20 21:19 ./Overview.ipynb
-rw-rw-rw- 1 root root      10914 Jun 20 21:19 ./About-the-Neptune-Notebook.ipynb
-rw-r--r-- 1 root root      21221 Aug 31 21:20 ./rgcn-gnn-embeddings-test.ipynb
-rw-r--r-- 1 root root  888463172 Sep  1 03:23 ./gnn_embeddings.csv
-rw-r--r-- 1 root root      93137 Sep  1 18:35 ./MovieSearchFull.ipynb
-rw-r--r-- 1 root root   30819722 Sep  1 18:37 ./xmovie.csv
-rw-r--r-- 1 root root    6662412 Sep  1 18:37 ./xmovie_tt.txt
-rw-r--r-- 1 root root    6662412 Sep  1 18:38 ./xsentence_tt.txt
-rw-r--r-- 1 root root 3045167080 Sep  1 18:38 ./xsentence_embeddings.csv

./01-Neptune-Database:
total 20
drwxrwxrwx 2 root root 6144 Jun 21 00:17 01-Getting-Started
drwxrwxrwx 2 root root 6144 Jun 21 00:17 02-Visualization
drwxrwxrwx 5 root root 6144 Jun 21 00:17 .
drwxrwxrwx 8 root root 6144 Jun 21 00:17 03-Sample-Applications
drwxr-xr-x 7 root root 6144 Sep  1 18:43 ..

./02-Neptune-Analytics:
total 20
drwxrwxrwx 2 root root 6144 Jun 21 0

In [18]:
%%bash

echo need to sort the files by ID allowing us to tie together by row number
#head -1 movies.csv > xmovie.csv
#head -1 sentence_embeddings.csv > xsentence_embeddings.csv
head -1 gnn_embeddings.csv > xgnn_embeddings.csv

#echo movie
#tail -n+2 movies.csv | sort -k1,1 -t"," -d >> xmovie.csv
#echo sentence
#tail -n+2 sentence_embeddings.csv | sort -k1,1 -t"," -d  >> xsentence_embeddings.csv
echo gnn
tail -n+2 gnn_embeddings.csv | sort -k1,1 -t"," -d  >> xgnn_embeddings.csv

echo confirm num lines the same
#wc -l xmovie.csv  xsentence_embeddings.csv
wc -l xmovie.csv xgnn_embeddings.csv xsentence_embeddings.csv


need to sort the files by ID allowing us to tie together by row number
gnn
confirm num lines the same
    646581 xmovie.csv
    690370 xgnn_embeddings.csv
    646581 xsentence_embeddings.csv
   1983532 total


In [21]:
%%bash

echo Check sort order of TT IDs is the same across the three files

#cat xmovie.csv | awk -F, '{print $1}' > xmovie_tt.txt
cat xgnn_embeddings.csv | awk -F, '{print $1}' > xgnn_tt.txt
#cat xsentence_embeddings.csv | awk -F, '{print $1}' > xsentence_tt.txt

echo diff movie gnn
diff xmovie_tt.txt xgnn_tt.txt | head -10

#echo diff movie sentence
#diff xmovie_tt.txt xsentence_tt.txt | head -10

echo diff gnn sentence
diff xgnn_tt.txt xsentence_tt.txt | head -10


Check sort order of TT IDs is the same across the three files
diff movie gnn
diff gnn sentence


In [30]:
!diff xmovie_tt.txt xsentence_tt.txt | head -10

In [31]:
!diff xmovie_tt.txt xgnn_tt.txt | head -10

In [32]:
!diff xgnn_tt.txt xsentence_tt.txt | head -10

In [38]:
movie_index_2 = {
    "settings": {
        "number_of_replicas": 1,
        "number_of_shards": 5,
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },  
    "mappings": {
        "properties": {
            "sentence_embedding": {
                "type": "knn_vector",
                "dimension": 384,
            },
            "gnn_embedding": {
                "type": "knn_vector",
                "dimension": 64,
            }
        }
    }
}

In [39]:
movie_sentence_index_2 = {
    "settings": {
        "number_of_replicas": 1,
        "number_of_shards": 5,
        "index.knn": True,
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "sentence_embedding": {
                "type": "knn_vector",
                "dimension": 384,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                }
            },
            "gnn_embedding": {
                "type": "knn_vector",
                "dimension": 64,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                }
            }
        }
    }
}


In [40]:
movie_gnn_index_2 = {
    "settings": {
        "number_of_replicas": 1,
        "number_of_shards": 5,
        "index.knn": True,
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "sentence_embedding": {
                "type": "knn_vector",
                "dimension": 384,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                }
            },
            "gnn_embedding": {
                "type": "knn_vector",
                "dimension": 64,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                }
            }
        }
    }
}


In [48]:
vs_aos_client.indices.delete(index="movie_2")
vs_aos_client.indices.create(index="movie_2",body=movie_index_2)

vs_aos_client.indices.delete(index="movie_gnn_2")
vs_aos_client.indices.create(index="movie_gnn_2",body=movie_gnn_index_2)

vs_aos_client.indices.delete(index="movie_sentence_2")
vs_aos_client.indices.create(index="movie_sentence_2",body=movie_sentence_index_2)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'movie_sentence_2'}

In [49]:
vs_aos_client.indices.get(index="movie_gnn_2")

{'movie_gnn_2': {'aliases': {},
  'mappings': {'properties': {'gnn_embedding': {'type': 'knn_vector',
     'dimension': 64,
     'method': {'engine': 'nmslib',
      'space_type': 'cosinesimil',
      'name': 'hnsw',
      'parameters': {}}},
    'sentence_embedding': {'type': 'knn_vector',
     'dimension': 384,
     'method': {'engine': 'nmslib',
      'space_type': 'cosinesimil',
      'name': 'hnsw',
      'parameters': {}}}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'movie_gnn_2',
    'knn': 'true',
    'creation_date': '1725227363941',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': 'PSeQr5EByQpUVyt3bu0g',
    'version': {'created': '136327827'}}}}}

In [50]:
vs_aos_client.indices.get(index="movie_2")

{'movie_2': {'aliases': {},
  'mappings': {'properties': {'gnn_embedding': {'type': 'knn_vector',
     'dimension': 64},
    'sentence_embedding': {'type': 'knn_vector', 'dimension': 384}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'movie_2',
    'creation_date': '1725227363541',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': '1pCQr5EBcxTHb9whbK-L',
    'version': {'created': '136327827'}}}}}

In [51]:
vs_aos_client.indices.get(index="movie_sentence_2")

{'movie_sentence_2': {'aliases': {},
  'mappings': {'properties': {'gnn_embedding': {'type': 'knn_vector',
     'dimension': 64,
     'method': {'engine': 'nmslib',
      'space_type': 'cosinesimil',
      'name': 'hnsw',
      'parameters': {}}},
    'sentence_embedding': {'type': 'knn_vector',
     'dimension': 384,
     'method': {'engine': 'nmslib',
      'space_type': 'cosinesimil',
      'name': 'hnsw',
      'parameters': {}}}}},
  'settings': {'index': {'number_of_shards': '2',
    'provided_name': 'movie_sentence_2',
    'knn': 'true',
    'creation_date': '1725227364398',
    'analysis': {'analyzer': {'default': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': '2JCQr5EBcxTHb9whcK8P',
    'version': {'created': '136327827'}}}}}

In [33]:
import json
from opensearchpy.helpers import bulk

def add_bulk_to_index(index_name, chunk):

    requests = []
    for rec in chunk:
        request = {
            "_op_type": "index",
            "_index": index_name,
            "title": (rec['fields'])['title'],
            #"year": (rec['fields'])['year'],
            #"averageRating": (rec['fields'])['averageRating'],
            #"numVotes": (rec['fields'])['numVotes'],
            #"runtime": (rec['fields'])['runtime'],
            "sentence_embedding": (rec['fields'])['sentence_embedding'],
            "gnn_embedding": (rec['fields'])['gnn_embedding']
        }
        request["id"] = rec['id']
        #print(request)

    requests.append(request)
    #print("\nBulk Load to Index")
    #print(len(requests))
    #print(requests)
    bulk(vs_aos_client, requests, max_chunk_bytes=(10 * 1024 * 1024))



In [52]:
import pandas as pd

CHUNK_SIZE=1000
movie_reader = pd.read_csv('xmovie.csv', iterator=True)
sentence_reader = pd.read_csv('xsentence_embeddings.csv', iterator=True)
gnn_reader = pd.read_csv('xgnn_embeddings.csv', iterator=True)

MOVIE_MAP = {
    'title:string': 'title',
    'year:int': 'year',
    'averageRating:float': 'averageRating',
    'numVotes:int': 'numVotes',
    'runtime:int': 'runtime'
}

def process_next_chunk():
    aos_chunks = []
    movie_df = None
    sentence_df = None
    gnn_df = None
    try:
        movie_df = movie_reader.get_chunk(CHUNK_SIZE)
        sentence_df = sentence_reader.get_chunk(CHUNK_SIZE)
        gnn_df = gnn_reader.get_chunk(CHUNK_SIZE)
        if len(movie_df) == 0:
            return False
    except StopIteration as e: 
        print("Stop")
        print(e)
        return False

    start_index = -1
    for index, row in movie_df.iterrows():
        if start_index == -1:
            start_index = index
            #print(start_index)
        aos_rec = {"id": row["~id"], 'fields': {}}
        aos_chunks.append(aos_rec)
        try:
            for m in MOVIE_MAP:
                if pd.isna(row[m]):
                    #skip
                    pass
                else:
                    aos_rec['fields'][MOVIE_MAP[m]]=row[m]
        except Exception as e:
            print(f"Error on movie on {index} {row}")
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
    for index, row in sentence_df.iterrows():
        try:
            aos_rec = aos_chunks[index-start_index]
            if row['~id']==aos_rec['id'] and 'embedding:vector' in row and len(row['embedding:vector']) > 0:
                aos_rec['fields']['sentence_embedding']=parse_str_embedding(row['embedding:vector'])
            else:
                raise Exception(f"Sentence Embedding/movie mismatch {str(index)} {str(row)} {str(aos_rec)}")
            #print(aos_rec['id'])
        except Exception as e:
            print(f"Error on {index} {row}")
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
    for index, row in gnn_df.iterrows():
        try:
            aos_rec = aos_chunks[index-start_index]
            if row['~id']==aos_rec['id'] and 'embedding:vector' in row and len(row['embedding:vector']) > 0:
                aos_rec['fields']['gnn_embedding']=parse_str_embedding(row['embedding:vector'])
            else:
                raise Exception(f"GNN Embedding/movie mismatch {str(index)} {str(row)} {str(aos_rec)}")
        except Exception as e:
            print(f"Error on {index} {row}")
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))

    #for idx in ['movie', 'movie_sentence', 'movie_gnn']:
    #print("\nDone process_next_chunk")
    print(len(aos_chunks))
    print(aos_chunks[0]['id'])
    #for idx in ['movie', 'movie_sentence']:
        #add_bulk_to_index(idx, aos_chunks)
    #print("call bulk load movie index")
    #print(len(aos_chunks))
    add_bulk_to_index('movie_2', aos_chunks)
    
    #print("call bulk load movie_sentence index")
    #print(len(aos_chunks))
    add_bulk_to_index('movie_sentence_2', aos_chunks)
    
    add_bulk_to_index('movie_gnn_2', aos_chunks)

    return True

while True:
    ret = process_next_chunk()
    #print("\nDone")
    #print(ret)
   
    if not ret:
        break

#ret = process_next_chunk()
#print("\nDone")
#print(ret)


1000
tt0000009
1000
tt0005210
1000
tt0007093
1000
tt0008543
1000
tt0009855
1000
tt0011176
1000
tt0012408
1000
tt0013605
1000
tt0014813
1000
tt0016005
1000
tt0017217
1000
tt0018418
1000
tt0019584
1000
tt0020838
1000
tt0022177
1000
tt0023525
1000
tt0024813
1000
tt0026114
1000
tt0027406
1000
tt0028621
1000
tt0029906
1000
tt0031178
1000
tt0032409
1000
tt0033676
1000
tt0034957
1000
tt0036185
1000
tt0037427
1000
tt0038661
1000
tt0039899
1000
tt0041269
1000
tt0042564
1000
tt0043825
1000
tt0045082
1000
tt0046343
1000
tt0047611
1000
tt0048939
1000
tt0050166
1000
tt0051412
1000
tt0052668
1000
tt0053904
1000
tt0055140
1000
tt0056352
1000
tt0057589
1000
tt0058951
1000
tt0060328
1000
tt0061757
1000
tt0063110
1000
tt0064527
1000
tt0065868
1000
tt0067278
1000
tt0068681
1000
tt0070120
1000
tt0071545
1000
tt0073022
1000
tt0074528
1000
tt0076019
1000
tt0077517
1000
tt0079019
1000
tt0080495
1000
tt0082009
1000
tt0083351
1000
tt0084852
1000
tt0086356
1000
tt0087942
1000
tt0089562
1000
tt0091129
1000
tt009